## MLOps Tools: MLflow and Hugging Face

> https://www.coursera.org/learn/mlops-mlflow-huggingface-duke/home/week/1

---

### Introduction

In [23]:
import os
from random import choice
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifact

In [6]:
log_param("threshold", 3)
log_param("verbosity", "DEBUG")

'DEBUG'

In [3]:
log_metric("timestamp", 3000)
log_metric("TTC", 33)

In [5]:
log_artifact('./data/test.txt')

For the next task we would like to track some random metrics

Let us first  create an experiment:

```
> mlflow experiments create --experiment-name log-metrics-demo
> Created experiment 'log-metrics-demo' with id 384253852139107280
```



In [7]:
mlflow.set_experiment("log-metrics-demo")

<Experiment: artifact_location='file:///Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn%20MLflow/MLFLow%20Course%20-%20Duke%20University/mlruns/384253852139107280', creation_time=1690609937121, experiment_id='384253852139107280', last_update_time=1690609937121, lifecycle_stage='active', name='log-metrics-demo', tags={}>

In [9]:
# lets log some metrics
metric_names = ["cpu", "ram", "disk"]
percentages = [i for i in range(0, 101)]

for i in range(10):
    print (choice(metric_names), choice(percentages))
    log_metric(key=choice(metric_names), value=choice(percentages))

disk 87
disk 78
disk 34
ram 77
disk 82
disk 19
ram 51
cpu 47
disk 47
cpu 47


Basically what happened here is
- we created a new experiment
- we set this experiment in mlflow
- we logged a bunch of metrics under this experiment

So if you see in /mlruns, there should be a folder corresponding to the id of the experiment we just created

![](https://imgur.com/En296nb.png)

In [17]:
# if an active run exists, end it
if mlflow.active_run():
    mlflow.end_run()

In [18]:
mlflow.set_experiment("log-metrics-demo")

<Experiment: artifact_location='file:///Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn%20MLflow/MLFLow%20Course%20-%20Duke%20University/mlruns/384253852139107280', creation_time=1690609937121, experiment_id='384253852139107280', last_update_time=1690609937121, lifecycle_stage='active', name='log-metrics-demo', tags={}>

In [19]:
# lets log some metrics
metric_names = ["cpu"]
percentages = [i for i in range(0, 101)]

with mlflow.start_run(run_name="cpu_new"):

    for i in range(400):
        # print (choice(metric_names), choice(percentages))
        log_metric(key="cpu_new", value=choice(percentages))

mlflow.end_run()

We can also query the metrics using the mlflow UI search box:

![](https://imgur.com/GszAvYj.png)

### Parameters, Version, Artifacts and Metrics

> https://github.com/databricks/mlflow-example-sklearn-elasticnet-wine/blob/master/train.py

---

In [21]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [22]:
np.random.seed(40)

In [24]:
# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
wine_path = "/Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn MLflow/MLFLow Course - Duke University/data/wine-quality.csv"
data = pd.read_csv(wine_path)

In [25]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [26]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

print (train.shape, test.shape)

(3673, 12) (1225, 12)


In [27]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


In [40]:
alpha, l1_ratio = 0.1, 0.2

In [ ]:
mlflow.create_experiment(name='wine-quality-experiment')

In [31]:
mlflow.set_experiment("wine-quality-experiment")

<Experiment: artifact_location='file:///Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn%20MLflow/MLFLow%20Course%20-%20Duke%20University/mlruns/562967201650279680', creation_time=1690620239059, experiment_id='562967201650279680', last_update_time=1690620239059, lifecycle_stage='active', name='wine-quality-experiment', tags={}>

In [42]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.100000, l1_ratio=0.200000):
  RMSE: 0.7818770443445561
  MAE: 0.6133216811213634
  R2: 0.21041880316513917


/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


__Parameters__:

Parameters represent the input settings or configurations of your machine learning model or script. They are the variables that you set before running your code and remain constant throughout the run. For example, parameters can include hyperparameters like learning rate, number of hidden units in a neural network, regularization strength, etc.

__Metrics__:

Metrics are the numerical values that you want to track to evaluate the performance of your model during training and testing. These can include loss, accuracy, precision, recall, F1 score, etc. Metrics are not fixed before running the code; they are computed during the run and may change for different runs, especially for different model configurations.




Next we registered the model on mlflow, lets load that in and run inference on it

In [43]:
import mlflow.pyfunc

In [47]:
# Load the model from the Model Registry (change <MODEL_NAME> with the actual name)
model = mlflow.pyfunc.load_model("models:/elasticnet/none")

In [48]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 01687e666c1d4f1a87ad43998b83f668

In [49]:
type(model)

mlflow.pyfunc.PyFuncModel

In MLflow, `mlflow.pyfunc.PyFuncModel` is a type of model that represents a generic Python function for model inference. It allows you to package and serve models in a way that is agnostic to the underlying machine learning library or framework. This means you can log and load models trained with various machine learning libraries (e.g., scikit-learn, TensorFlow, PyTorch) as `PyFuncModel` and use a consistent API for inference.

Here are some key points about `mlflow.pyfunc.PyFuncModel` and how it differs from a scikit-learn model:

1. **Model Agnostic**: `PyFuncModel` is model-agnostic, meaning it can encapsulate models trained with different libraries. It enables you to log and load models without being tightly coupled to a specific machine learning framework.

2. **Serialization**: When you log a model with MLflow, it serializes the model and its artifacts to a format that can be easily loaded and used for inference later. For scikit-learn models, MLflow serializes the model using pickle. For other libraries like TensorFlow or PyTorch, MLflow uses appropriate serialization methods (e.g., TensorFlow's SavedModel format, PyTorch's .pt file).

3. **Inference**: To make predictions with a `PyFuncModel`, you can use the `predict()` method. The input data should match the format expected by the underlying model. In the example provided in the previous answer, we assumed a Pandas DataFrame as the input, but the format may differ depending on the model's requirements.

4. **Flexibility**: Using `PyFuncModel` allows you to easily switch between different machine learning libraries without changing your inference code. You can load and use TensorFlow, PyTorch, or scikit-learn models in the same way, provided they are saved as `PyFuncModel`.

5. **Deployment**: Since `PyFuncModel` is a generic Python function, you can easily deploy it for inference using different deployment solutions, like REST APIs, web services, serverless functions, or containerized environments.

In contrast, a scikit-learn model (`sklearn.base.BaseEstimator`) is specific to the scikit-learn library. While it is a widely-used and powerful library for machine learning, it means that you're limited to using only scikit-learn models if you use the `BaseEstimator` type.

With `mlflow.pyfunc.PyFuncModel`, MLflow provides a more flexible and interoperable way of handling models, which can be advantageous in projects that involve different machine learning libraries or when you want to decouple your inference code from the model's training framework.

In [50]:
predicted_qualities_loaded_model = model.predict(test_x)

In [53]:
np.sum(predicted_qualities_loaded_model)

7196.524270453067

In [54]:
np.sum(predicted_qualities)

7196.524270453067